In [1]:
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Weaviate
from langchain_openai import ChatOpenAI

import weaviate
from dotenv import load_dotenv,find_dotenv
from weaviate.embedded import EmbeddedOptions
from langchain_community.document_loaders import PyPDFLoader

from langchain_core.runnables import (
    RunnableParallel,
    RunnablePassthrough
)
from langchain.schema.output_parser import StrOutputParser

import os
import sys


In [2]:
OPENAI_KEY=os.environ["OPENAI_APIKEY"]


In [3]:
sys.path.append('..')

from backend.rag_pipeline import *

/home/kerod/Desktop/week_11/RAG-system-for-Contract-Q-A/week_11/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
loader = PyPDFLoader("../data/RaptorContract.pdf")

data = data_loader(loader)

In [5]:
docs = text_splitter(data)

In [6]:
import weaviate

client = weaviate.Client(embedded_options=EmbeddedOptions)


/home/kerod/Desktop/week_11/RAG-system-for-Contract-Q-A/week_11/lib/python3.10/site-packages/weaviate/warnings.py:158: DeprecationWarning: Dep016: You are using the Weaviate v3 client, which is deprecated.
            Consider upgrading to the new and improved v4 client instead!
            See here for usage: https://weaviate.io/developers/weaviate/client-libraries/python
            
  warnings.warn(


Started /home/kerod/.cache/weaviate-embedded: process ID 60204


{"action":"startup","default_vectorizer_module":"none","level":"info","msg":"the default vectorizer modules is set to \"none\", as a result all new schema classes without an explicit vectorizer setting, will use this vectorizer","time":"2024-03-02T02:09:31+03:00"}
{"action":"startup","auto_schema_enabled":true,"level":"info","msg":"auto schema enabled setting is set to \"true\"","time":"2024-03-02T02:09:31+03:00"}
{"level":"info","msg":"No resource limits set, weaviate will use all available memory and CPU. To limit resources, set LIMIT_RESOURCES=true","time":"2024-03-02T02:09:32+03:00"}
{"level":"warning","msg":"Multiple vector spaces are present, GraphQL Explore and REST API list objects endpoint module include params has been disabled as a result.","time":"2024-03-02T02:09:32+03:00"}
{"action":"grpc_startup","level":"info","msg":"grpc server listening at [::]:50060","time":"2024-03-02T02:09:32+03:00"}
{"action":"restapi_management","level":"info","msg":"Serving weaviate at http://12

In [7]:
from langchain.retrievers.weaviate_hybrid_search import WeaviateHybridSearchRetriever

In [8]:
retriever = WeaviateHybridSearchRetriever(
    client=client,
    alpha= 0,
    index_name="LangChain",
    text_key="text",
    attributes=[],
    create_schema_if_missing=True,
)

In [9]:
retriever.add_documents(docs)

{'error': [{'message': 'update vector: connection to: OpenAI API failed with status: 429 error: You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.'}]}
{'error': [{'message': 'update vector: connection to: OpenAI API failed with status: 429 error: You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.'}]}
{'error': [{'message': 'update vector: connection to: OpenAI API failed with status: 429 error: You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.'}]}
{'error': [{'message': 'update vector: connection to: OpenAI API failed with status: 429 error: You exceeded your current qu

['48f71350-df73-4fef-b7a6-85ba1b976aa5',
 '741762d3-6395-4a02-b8bf-2423f5f2da4d',
 'f9815ba3-383d-4642-b858-22e0ce2c4629',
 'bf0034f5-f792-4d42-8de9-bf0151420db3',
 '426c5b83-2287-4640-b0ca-bb8edcdb0b8d',
 '63735066-a9b0-4e0f-81c2-a89b97b7b8e9',
 '8f50aaba-e65a-4f9b-86c4-0cd5388bd798',
 'baaff781-8eca-4749-b150-998d79ff0eea',
 '14e6f1a2-726d-4257-bf04-b072f29282ea',
 'de8af96e-4973-4140-b73a-00ba17faed60',
 '1c735e83-7d61-4764-a767-09fd30a10b0b',
 '22bf46fa-c8be-4197-86ce-fdd7a517be67',
 'e42b328b-f317-4efe-818f-250cf97751c7',
 '8650a5ee-6654-4eae-b5ac-8db03fcf1662',
 'fc0b7bfb-dd00-42cd-8526-ddf89ed87187',
 '43ff2773-d24b-41c2-9036-4c193dbce8f2',
 'cca5e7f1-ff37-43a8-a8d4-101ad8de51d6',
 '30b7baba-29e1-4a76-9196-d6018392da6c',
 '759b1c65-5b5d-4a29-8994-863c0d2af89c',
 '69cc498c-ad6e-4cdb-bf70-1b82f620d174',
 '2e4a0967-8dc8-4dc7-b9c2-7ab19c92c2dd',
 '9c85c710-ca93-48d3-829a-2443293aa441',
 'f307877d-8ca8-4851-b4e6-d160db1643ce',
 '571fb6f9-046f-4633-8119-ab4655b6408d',
 'dcb304f7-413d-

In [10]:
retriever.get_relevant_documents("what is the Escrow Amount?")

[Document(page_content='agreed\nto\nby\nthe\nparties\nand\nthe\ndecision\nof\nthe\nAccounting\nFirm\npursuant\nto\nthis\nSection\n2.07\nis \nreferred\nto\nherein\nas\nthe\n“\nFinal\nClosing\nStatement\n”.\nEach\nof\nthe\nparties\nto\nthis\nAgreement\nagrees \nto\nuse\nits\ncommercially\nreasonable\nefforts\nto\ncooperate\nwith\nthe\nAccounting\nFirm\n(including\nby \nexecuting\na\ncustomary\nengagement\nletter\nreasonably\nacceptable\nto\nit)\nand\nto\ncause\nthe \nAccounting\nFirm\nto\nresolve\nany\nsuch\ndispute\nas\nsoon\nas\npracticable\nafter\nthe\ncommencement\nof \nthe\nAccounting\nFirm’ s\nengagement.\n(e)\nPurchase\nPrice\nAdjustment\n.\nIf\nany\nof\nthe\nClosing\nCash\nAmount,\nthe\nClosing\nDebt \nAmount\nor\nthe\nSeller\nTransaction\nExpenses\n(as\nfinally\ndetermined\npursuant\nto\nthis\nSection\n2.07 \nand\nas\nset\nforth\nin\nthe\nFinal\nClosing\nStatement)\ndiffers\nfrom\nthe\nestimated\namounts\nthereof\nset \nforth\nin\nthe\nEstimated\nClosing\nStatement,\nthe\nPurcha

In [11]:

def create_chians(retriever):
        # Define LLM
    llm = ChatOpenAI(openai_api_key = OPENAI_KEY,model_name="gpt-3.5-turbo", temperature=0)

    # Define prompt template
    prompt_template = """You are an assistant for question-answering tasks. 
    Use the following pieces of retrieved context to answer the question. 
    If you don't know the answer, just say that you don't know. 
    Question: {question} 
    Context: {context} 
    Answer:
    """

    prompt = ChatPromptTemplate.from_template(prompt_template)


    retrieval = RunnableParallel(
        {"context": retriever,  "question": RunnablePassthrough()} 
    )

    chain = retrieval | prompt | llm | StrOutputParser()

    return chain

In [12]:
chain = create_chians(retriever)

In [14]:
answer = chain.invoke("what does \"Escrow Amount\" mean?")

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [ ]:
answer

'The "Escrow Amount" refers to the amount that the Buyer will deposit in escrow on behalf of the Sellers at closing, as outlined in the Escrow Agreement. This amount will be held and released to the Company Securityholders according to the terms of the Escrow Agreement, with the Securityholders sharing in the released amounts based on their Pro Rata Percentages.'